In [4]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh

In [5]:
import fiftyone as fo

datasets = fo.list_datasets()
print("Existing Datasets:", datasets)


Existing Datasets: ['Voxel51/VisDrone2019-DET']


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import fiftyone as fo
dataset = fo.load_dataset("Voxel51/VisDrone2019-DET")  #"Voxel51/VisDrone2019-DET"

In [8]:
print(f"Dataset name: {dataset.name}")
print(f"Number of samples: {len(dataset)}")

for sample in dataset.take(5):
    print(sample.id)


Dataset name: Voxel51/VisDrone2019-DET
Number of samples: 8629
66182184534f2808b7fb6221
66182184534f2808b7fb7e42
66182184534f2808b7fb764b
66182184534f2808b7fb6701
66182184534f2808b7fb7f2e


In [9]:
first_sample = dataset.first()
field_names = first_sample.to_dict().keys()

print("Field names in the dataset:", list(field_names))


Field names in the dataset: ['filepath', 'tags', 'metadata', 'created_at', 'last_modified_at', 'ground_truth']


In [ ]:
for sample in dataset:
    image_path = sample.filepath
    print(f"Image Path: {image_path}")

    if sample.ground_truth is not None:
        detections = sample.ground_truth.detections
        for detection in detections:
            print(f"Label: {detection.label}, Bounding Box: {detection.bounding_box}")
    else:
        print("No ground truth annotations available.")


In [ ]:
import cv2

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image at {image_path}. Skipping...")
        return None

    img = cv2.resize(img, (300, 300))
    img = img / 255.0
    return img

images = []
labels = []

for sample in dataset:
    image_path = sample.filepath
    img = preprocess_image(image_path)

    if img is not None:
        images.append(img)

        if sample.ground_truth is not None:
            detections = sample.ground_truth.detections
            label = [detection.label for detection in detections]
            labels.append(label)
        else:
            labels.append([])

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(labels)

print(f"Encoded labels shape: {y_encoded.shape}")


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

print(f"Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG16

def ssd_model(input_shape=(300, 300, 3), num_classes=20):

    base_model = VGG16(input_shape=input_shape, include_top=False, weights='imagenet')

    feature_maps = [
        base_model.get_layer('block3_conv3').output,
        base_model.get_layer('block4_conv3').output,
        base_model.get_layer('block5_conv3').output,
        base_model.output  # 10x10
    ]


    x = base_model(input_shape)
    predictions = []


    for feature_map in feature_maps:
        x = layers.Conv2D(4 * num_classes, kernel_size=(3, 3), padding='same')(feature_map)
        predictions.append(x)

        x = layers.Conv2D(4, kernel_size=(3, 3), padding='same')(feature_map)
        predictions.append(x)


    loc_preds = tf.concat([predictions[i] for i in range(0, len(predictions), 2)], axis=1)
    class_preds = tf.concat([predictions[i] for i in range(1, len(predictions), 2)], axis=1)


    model = Model(inputs=base_model.input, outputs=[loc_preds, class_preds])
    return model

model = ssd_model()
model.compile(optimizer='adam',
              loss={'bbox_output': 'mean_squared_error', 'class_output': 'sparse_categorical_crossentropy'},
              metrics=['accuracy'])

model.summary()


In [ ]:
num_samples = 1000
X_train = np.random.random((num_samples, 300, 300, 3))
y_train_boxes = np.random.random((num_samples, 4))
y_train_classes = np.random.randint(0, 20, num_samples)

model.fit(X_train, [y_train_boxes, y_train_classes], epochs=10, batch_size=32)

loss = model.evaluate(X_train, [y_train_boxes, y_train_classes])
print(f"Test Accuracy: {test_acc}")

In [2]:
predictions = model.predict(X_train)

NameError: name 'model' is not defined